In [1]:
# Import Libraries and Tools
import os
import sys
from astropy.io import fits
import sunpy.map
from aiapy.calibrate import register, update_pointing
import glob
import numpy as np
import matplotlib.pyplot as plt

# ACWE utilities
# Root directory of the project
ROOT_DIR = os.path.abspath("../../../../../")

# Import ACWE Tools
sys.path.append(ROOT_DIR)
from ACWE_python_fall_2023 import acweSaveSeg_v5

In [ ]:
# Key Variables

# Dataset
dataset = '/home/jgra/Coronal Holes/newDataset/'

# ACWE Folders
SaveFolderMag = '/mnt/coronal_holes/Paper 2/Code 02 Observations/'
SaveFolderMag = SaveFolderMag + 'HMI_Experiments/TestEvolutionMethods/Standard/'
SaveFolderEUV = '/mnt/coronal_holes/Paper 1/Code Paper I Observations/Standard/'

# ACWE Data
magMethod  = 'Uni' # 'FluxImb' 'Homogentiy' 'Skew' 
ACWEprefix = 'ACWE.'
acweChoice = '193'

In [ ]:
# Open File and Prepare for ACWE
def openAIA(filename):
    
    # Extract Image and Header Data
    hdulist = fits.open(filename)
    hdulist.verify('silentfix') # no clue why this is needed for successful data read
    h = hdulist[1].header
    J = hdulist[1].data
    hdulist.close()
    
    # Update to Level 1.5 Data Product
    if h['LVL_NUM'] < 1.5:
        m = sunpy.map.Map((J,h))    # Create Sunpy Map
        m = update_pointing(m)      # Update Header based on Latest Information
        m_registrered = register(m) # Recenter and rotate to Solar North
        I = m_registrered.data
        # Undo Keword Renaming
        H = dict()
        for k in m_registrered.meta.keys(): 
            H[k.upper()] = m_registrered.meta[k] 
    # Skip if already Level 1.5
    else:
        I = J*1
        H = dict(h)
        
        # Prepare Display Version
    Idsp = np.clip(I,20,2500)
    Idsp = np.log10(Idsp)
    Idsp = Idsp - np.min(Idsp)
    Idsp = Idsp/np.max(Idsp)
    
    return I,Idsp,H

In [ ]:
# Ensure Folder Exists
saveDirectory = '/Figures/MagScaleConstancy/' + magMethod + '/AreaCheck/'
saveDirectory = ROOT_DIR + saveDirectory

if not os.path.exists(saveDirectory):
    os.makedirs(saveDirectory)

In [ ]:
# Prepare for loop

# Get Dataset
segMagFiles = sorted(glob.glob(SaveFolderMag + magMethod + '/*/*/' + ACWEprefix + '*.npz'))

In [ ]:
# Loop test

# Prepare for loop
CR = ''

# cycle through files
for i in range(len(segMagFiles)):
    
    segMileFile = segMagFiles[i]
    
    # Inform user as to Carrington Rotation
    currentCR = segMileFile.split('/')[-3]
    if CR != currentCR:
        CR = str(currentCR)
        print()
        print('------')
        print(CR + ':')
        newsaveDirectory = saveDirectory + CR + '/'
    
    # Open Files
    _,MAH,MagSeg = acweSaveSeg_v5.openSeg(segMileFile)
    
    # File Shape
    segShape = np.array(MagSeg.shape)
    
    # Check files with large area identified as CH
    if np.sum(MagSeg.astype(int)) / (segShape[0] * segShape[1]) >= 0.025:
        
        if not os.path.exists(newsaveDirectory):
            os.makedirs(newsaveDirectory)

        # Find original EUV image
        euvFile  = os.path.basename(segMileFile).split(ACWEprefix)[1].split('.fits')[0]
        euvFile  = glob.glob(dataset+'*/*/*' + euvFile + '*')[0]
        _,Idsp,H = openAIA(euvFile)
        
        # Open Segmentations
        segEUVfile = os.path.basename(segMileFile).split(ACWEprefix)[1].split('.fits')[0]
        segEUVfile = glob.glob(SaveFolderEUV+'*/*/*' + segEUVfile + '*')[0]
        _,EAH,EUVseg = acweSaveSeg_v5.openSeg(segEUVfile)
        
        # Show Resutls
        plt.figure(figsize=[5,2],dpi=300)
        plt.rcParams.update({'font.size': 8})
        title = H['T_REC']
        plt.suptitle(title)
        plt.rcParams.update({'font.size': 5})
        plt.subplot(1,3,1)
        plt.imshow(np.flip(Idsp,axis=0),cmap='gray')
        plt.axis(False)
        plt.title('EUV Observation')
        plt.subplot(1,3,2)
        plt.imshow(np.flip(EUVseg,axis=0),interpolation='None')
        plt.axis(False)
        plt.title('EUV Segmentation')
        plt.subplot(1,3,3)
        plt.imshow(np.flip(MagSeg,axis=0),interpolation='None')
        plt.axis(False)
        plt.title('EUV+HMI Segmentation')
        title = newsaveDirectory + title.replace(':','') + '.png'
        plt.savefig(title)
        plt.show()

        plt.subplot(1,2,1)
        plt.imshow(np.flip(EAH['INIT_MASK'],axis=0),interpolation='None')
        plt.title('Seed - EUV')
        plt.subplot(1,2,2)
        plt.imshow(np.flip(MAH['INIT_MASK'],axis=0),interpolation='None')
        plt.title('Seed - EUV+HMI')
        plt.show()
        print()
        print()